In [ ]:
import copy
import sys

import numpy as np
from numpy.random import default_rng
import pydicom

import pymedphys
from pymedphys._dicom import create

# Create two 100 × 100 × 100 DICOM dose grids:

In [ ]:
WIDTH=3

DOSE_GRID_SHAPE = (WIDTH, WIDTH, WIDTH)

SCALE = 1
BITS_ALLOCATED = 32

rng = default_rng()

data_ref = rng.normal(loc=200, scale=4, size=DOSE_GRID_SHAPE).astype(np.float32) # 2% RSU
data_eval = rng.normal(loc=200, scale=4, size=DOSE_GRID_SHAPE).astype(np.float32)

print(type(data_ref))
# print(data_ref.shape)

meta = pydicom.Dataset()
meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

rtdose_dict = {
    "PatientID": "PMP",
    "Modality": "RTDOSE",
    "ImagePositionPatient": [-DOSE_GRID_SHAPE[2]+1, -DOSE_GRID_SHAPE[1]+1, -DOSE_GRID_SHAPE[0]+1],
    "ImageOrientationPatient": [1, 0, 0, 0, 1, 0],
    "BitsAllocated": BITS_ALLOCATED,
    "Rows": DOSE_GRID_SHAPE[1],
    "Columns": DOSE_GRID_SHAPE[2],
    "PixelRepresentation": 0,
    "SamplesPerPixel": 1,
    "PhotometricInterpretation": "MONOCHROME2",
    "PixelSpacing": [2.0, 2.0],
    "GridFrameOffsetVector": np.arange(0, 2 * DOSE_GRID_SHAPE[0], 2).tolist(),
    "DoseGridScaling": SCALE,
    "DoseSummationType": "PLAN",
    "DoseType": "PHYSICAL",
    "DoseUnits": "GY",
}

ds_ref = create.dicom_dataset_from_dict(rtdose_dict)
ds_ref.file_meta = meta
ds_ref.is_implicit_VR = True
# ds_ref['FloatPixelData'].VR = 'OB'
ds_ref.fix_meta_info(enforce_standard=False)

ds_ref.FloatPixelData = data_ref.tobytes()

# print(ds_ref.pixel_array)
# print(ds_ref.pixel_array.shape)

ds_eval = copy.deepcopy(ds_ref)

ds_eval.FloatPixelData = data_eval.tobytes()

# print(ds_eval.pixel_array)
# print(ds_eval.pixel_array.shape)


# Gamma

In [ ]:
axes_ref, dose_ref = pymedphys.dicom.zyx_and_dose_from_dataset(ds_ref)
axes_eval, dose_eval = pymedphys.dicom.zyx_and_dose_from_dataset(ds_eval)

print(axes_ref)

dose_percent_thresh = 2
dist_mm_thresh = 2

In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_ref, dose_ref, 
    axes_eval, dose_eval, 
    dose_percent_thresh, dist_mm_thresh, interpolator="scipy")

In [ ]:
%%time

gamma = pymedphys.gamma(
    axes_ref, dose_ref, 
    axes_eval, dose_eval, 
    dose_percent_thresh, dist_mm_thresh, interpolator="econforge")